## 1) BoardingData.csv
## 2) FrequentFlyerForum-Profiles.json
## 3) PointzAggregator-AirlinesData.xml
## 4) Sirena-export-fixed.tab
## 5) Skyteam_Timetable.pdf
## 6) SkyTeam-Exchange.yaml
## 7) YourBoardingPassDotAero.zip

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import yaml
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm

# 1) BoardingData.csv

In [2]:
boarding_data = pd.read_csv("BoardingData.csv", delimiter=";")
boarding_data.head()

,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
0,SAVELII,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAI,I.,NIKOLSKII,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLII,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow


In [3]:
print("Количество записей в таблице: {}".format(len(boarding_data)))
print("Количество уникальных пассажиров по их номеру паспорта: {}".format((len(np.unique(boarding_data['PassengerDocument'])))))
print("Количество уникальных направлений: {}".format((len(np.unique(boarding_data['Destination'])))))

Количество записей в таблице: 155147
Количество уникальных пассажиров по их номеру паспорта: 73263
Количество уникальных направлений: 48


In [4]:
print("В какие направления летают люди:")
print(list(np.unique(boarding_data['Destination'])))
print()
print("Можно заметить, что здесь города только России")

В какие направления летают люди:
['Abakan', 'Anapa', 'Arkhangelsk', 'Astrakhan', 'Barnaul', 'Belgorod', 'Chelyabinsk', 'Irkutsk', 'Kaliningrad', 'Kazan', 'Kemerovo', 'Khabarovsk', 'Khanty', 'Komsomolsk', 'Krasnodar', 'Krasnoyarsk', 'Magadan', 'Magnitogorsk', 'Makhachkala', 'Moscow', 'Murmansk', 'Nizhnekamsk', 'Nogliki', 'Novokuznetsk', 'Novosibirsk', 'Novyj Urengoj', 'Okha', 'Omsk', 'Orenburg', 'Perm', 'Petropavlovsk', 'Rostov', 'Salekhard', 'Samara', 'Saransk', 'Saratov', 'Sochi', 'Stavropol', 'Surgut', 'Syktyvkar', 'Tomsk', 'Tyumen', 'Ufa', 'Vladivostok', 'Volgograd', 'Voronezh', 'Yekaterinburg', 'Yuzhno']

Можно заметить, что здесь города только России


In [5]:
print("За какой период времени приведены данные:")
dates = sorted(list(set(boarding_data["FlightDate"])))
print("Первая дата: {}".format(dates[0]))
print("Последняя дата: {}".format(dates[-1]))

За какой период времени приведены данные:
Первая дата: 2017-01-01
Последняя дата: 2018-01-01


# 2) FrequentFlyerForum-Profiles.json 
### видимо это профили этих людей на форуме
#### заметим, что разных профилей здесь 66592
* departure - это откуда человек стартовал
* arrival - это куда человек прилетел

In [6]:
with open('FrequentFlyerForum-Profiles.json') as json_file:
    frequent_flyer_forum = json.load(json_file)

In [7]:
frequent_flyer_forum['Forum Profiles'][9]

{'Registered Flights': [{'Date': '2017-07-30',
   'Codeshare': False,
   'Arrival': {'City': 'Shanghai', 'Airport': 'PVG', 'Country': 'China'},
   'Flight': 'MU5606',
   'Departure': {'City': 'Hami', 'Airport': 'HMI', 'Country': 'China'}},
  {'Date': '2017-01-17',
   'Codeshare': False,
   'Arrival': {'City': 'Almaty', 'Airport': 'ALA', 'Country': 'Kazakhstan'},
   'Flight': 'SU1946',
   'Departure': {'City': 'Moscow',
    'Airport': 'SVO',
    'Country': 'Russian Federation'}},
  {'Date': '2017-05-05',
   'Codeshare': True,
   'Arrival': {'City': 'Shanghai', 'Airport': 'PVG', 'Country': 'China'},
   'Flight': 'MU8274',
   'Departure': {'City': 'Amsterdam',
    'Airport': 'AMS',
    'Country': 'Netherlands'}},
  {'Date': '2017-02-01',
   'Codeshare': False,
   'Arrival': {'City': 'Saint Petersburg',
    'Airport': 'LED',
    'Country': 'Russian'},
   'Flight': 'SU6562',
   'Departure': {'City': 'Sochi',
    'Airport': 'AER',
    'Country': 'Russian Federation'}},
  {'Date': '2017-12-23

In [8]:
# посмотрим, разные ли они все эти профили
nicknames = []
for i in range(len(frequent_flyer_forum['Forum Profiles'])):
    nickname = frequent_flyer_forum['Forum Profiles'][i]['NickName']
    nicknames.append(nickname)
    
print("Количество уникальных никнеймов на форуме в данных: {}".format(len(frequent_flyer_forum['Forum Profiles'])))
assert len(nicknames) == len(set(nicknames)) == len(frequent_flyer_forum['Forum Profiles'])

Количество уникальных никнеймов на форуме в данных: 66592


In [9]:
print("Поля в одном профиле пользователя:")
print(list(frequent_flyer_forum['Forum Profiles'][0].keys()))

Поля в одном профиле пользователя:
['Registered Flights', 'NickName', 'Travel Documents', 'Sex', 'Loyality Programm', 'Real Name']


In [10]:
first_person = frequent_flyer_forum['Forum Profiles'][0]
first_person_flights = first_person["Registered Flights"]
# Посмотрим, записи о каких странах ОТПРАВЛЕНИЯ содержатся в 'Registered Flights' первого пользователя
print(set(list(map(lambda flight: flight["Arrival"]["Country"], first_person_flights))))
print()
print()
# Посмотрим, записи о каких странах ПРИБЫТИЯ содержатся в 'Registered Flights' первого пользователя
print(set(list(map(lambda flight: flight["Departure"]["Country"], first_person_flights))))

{'United States', 'Poland', 'United States of', 'France'}


{'Poland', 'France', 'United States', 'Russian Federation', 'United States of'}


In [11]:
len(frequent_flyer_forum)

1

In [12]:
frequent_flyer_forum['Forum Profiles'][0].keys()

dict_keys(['Registered Flights', 'NickName', 'Travel Documents', 'Sex', 'Loyality Programm', 'Real Name'])

In [13]:
frequent_flyer_forum['Forum Profiles'][0]['Real Name']

{'Last Name': None, 'First Name': None}

In [14]:
# first_names = []
# last_names = []

# 3) PointzAggregator-AirlinesData.xml

In [15]:
# Распаршен в отдельном файле

# 4) Sirena-export-fixed.tab

In [16]:
with open("Sirena-export-fixed.tab", 'r', encoding="utf8") as sirena_export_fixed:
    line = sirena_export_fixed.readline().strip()
print(line)

PaxName                                                     PaxBirthDateDepartDate  DepartTime  ArrivalDate ArrivalTime FlightCodeShFrom  Dest  Code  e-Ticket          TravelDoc   Seat  Meal  TrvClsFare  Baggage     PaxAdditionalInfo                                           AgentInfo


In [17]:
indexes = (0, 60, 72, 84, 96, 108, 120, 126, 132, 138, 144, 150, 168, 180, 186, 192, 198, 204, 216, 276, None)

In [18]:
list(zip(indexes, indexes[1:]))

[(0, 60),
 (60, 72),
 (72, 84),
 (84, 96),
 (96, 108),
 (108, 120),
 (120, 126),
 (126, 132),
 (132, 138),
 (138, 144),
 (144, 150),
 (150, 168),
 (168, 180),
 (180, 186),
 (186, 192),
 (192, 198),
 (198, 204),
 (204, 216),
 (216, 276),
 (276, None)]

In [19]:
tuple(
    line[begin:end].strip()
    for begin, end in zip(indexes, indexes[1:])
)

('PaxName',
 'PaxBirthDate',
 'DepartDate',
 'DepartTime',
 'ArrivalDate',
 'ArrivalTime',
 'Flight',
 'CodeSh',
 'From',
 'Dest',
 'Code',
 'e-Ticket',
 'TravelDoc',
 'Seat',
 'Meal',
 'TrvCls',
 'Fare',
 'Baggage',
 'PaxAdditionalInfo',
 'AgentInfo')

In [20]:
def line_parser(line):
    return tuple(
        line[begin:end].strip()
        for begin, end in zip(indexes, indexes[1:])
    )

In [21]:
with open("Written-sirena-export-fixed.csv", 'w', encoding="utf8") as write_file, \
     open("Sirena-export-fixed.tab", 'r', encoding="utf8") as read_file:
    for line in read_file:
        write_file.writelines(','.join(line_parser(line)) + '\n')

In [22]:
sirena_export_fixed = pd.read_csv("Written-sirena-export-fixed.csv", sep=',', encoding='utf8')
sirena_export_fixed.head()

,PaxName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,ZBQSPY,7360415302044672,9375 053270,NaN,NaN,J,JGRPGN,0PC,S FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,MHPBBX,7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,NaN,9 FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,REDLVB,5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,NaN,F,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,NSJNGQ,5274206497242737,98 6865148,NaN,NaN,J,JFLXLS,0PC,# FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,VWNYGF,6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


In [23]:
print("Количество записей в таблице: {}".format(len(sirena_export_fixed)))
print("Количество уникальных пассажиров по их номеру паспорта: {}".format((len(np.unique(sirena_export_fixed['TravelDoc'])))))
print("Количество уникальных направлений отправления: {}".format((len(np.unique(sirena_export_fixed['From'])))))
print("Количество уникальных направлений прибытия: {}".format((len(np.unique(sirena_export_fixed['Dest'])))))

Количество записей в таблице: 155757
Количество уникальных пассажиров по их номеру паспорта: 83217
Количество уникальных направлений отправления: 48
Количество уникальных направлений прибытия: 48


# 5) Skyteam_Timetable.pdf

# 6) SkyTeam-Exchange.yaml

In [24]:
import json

In [25]:
with open('sample.json') as json_file:
    skyteam_exchange = json.load(json_file)

In [26]:
# Весь год каждый день какие-то записи
print(list(skyteam_exchange.keys())[:15])
print()
print()
print(list(skyteam_exchange.keys())[-15:])

['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13', '2017-01-14', '2017-01-15']


['2017-12-18', '2017-12-19', '2017-12-20', '2017-12-21', '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31', '2018-01-01']


In [27]:
skyteam_exchange['2017-01-01']

{'AF1145': {'FF': {'FB 520518073': {'CLASS': 'Y', 'FARE': 'YRSTFN'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'CDG'},
 'AF1844': {'FF': {'FB 18427809': {'CLASS': 'Y', 'FARE': 'YGRPZR'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'CDG'},
 'AF4855': {'FF': {'KE 993927664': {'CLASS': 'Y', 'FARE': 'YRSTJQ'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'CDG'},
 'AF4901': {'FF': {'SU 789402704': {'CLASS': 'Y', 'FARE': 'YGRPAT'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'CDG'},
 'AZ545': {'FF': {'KE 352734424': {'CLASS': 'Y', 'FARE': 'YFLXKO'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'FCO'},
 'AZ595': {'FF': {'SU 667777811': {'CLASS': 'Y', 'FARE': 'YRSTXE'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'FCO'},
 'AZ7163': {'FF': {'SU 451734642': {'CLASS': 'A', 'FARE': 'AGRPXS'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'FCO'},
 'AZ7209': {'FF': {'SU 272025028': {'CLASS': 'J', 'FARE': 'JSTNDU'}},
  'FROM': 'SVO',
  'STATUS': 'LANDED',
  'TO': 'NAP'},
 'C

# 7) YourBoardingPassDotAero.zip

In [28]:
# тут какие то билеты на самолет на каждый день

# Для начала попробуем сджойнить файлы csv и tab

In [29]:
boarding_data.head()

,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
0,SAVELII,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAI,I.,NIKOLSKII,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLII,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow


In [30]:
boarding_data["PassengerFirstName"] = list(map(lambda x: x.lower(), boarding_data["PassengerFirstName"]))
boarding_data["PassengerSecondName"] = list(map(lambda x: x.lower(), boarding_data["PassengerSecondName"]))
boarding_data["PassengerLastName"] = list(map(lambda x: x.lower(), boarding_data["PassengerLastName"]))

In [31]:
boarding_data.head(3)

,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
0,savelii,viktorovich,rusanov,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,lev,markovich,isaev,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,nikolai,i.,nikolskii,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd


In [32]:
sirena_export_fixed.head()

,PaxName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,Code,e-Ticket,TravelDoc,Seat,Meal,TrvCls,Fare,Baggage,PaxAdditionalInfo,AgentInfo
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,ZBQSPY,7360415302044672,9375 053270,NaN,NaN,J,JGRPGN,0PC,S FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,MHPBBX,7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,NaN,9 FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,REDLVB,5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,NaN,F,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,NSJNGQ,5274206497242737,98 6865148,NaN,NaN,J,JFLXLS,0PC,# FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,VWNYGF,6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


In [33]:
rus2eng = \
{'р': 'r',
 'в': 'v',
 'е': 'e',
 'ж': 'zh',
 'х': 'kh',
 'ц': 'ts',
 'ч': 'ch',
 'ш': 'sh',
 'щ': 'shch',
 'ю': 'ju',
 'я': 'ja',
 'ш': 'sh',
 'щ': 'shch',
 'ё': 'e',
 'о': 'o',
 'ы': 'y',
 'з': 'z',
 'й': 'i', 
 'и': 'i',
 'б': 'b', 
 'с': 's',
 'а': 'a',
 'ь': "'",
 'л': 'l',
 "п": "p",
 'т': 't',
 'н': 'n',
 'м': 'm',
 "д": "d",
 " ": " ",
 "г": "g",
 "к": "k",
 "у": "u",
 "ф": "f",
 "э": "e"}

In [34]:
"".join(map(lambda x: rus2eng.get(x, ''), list("ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ".lower())))

"ozerov il'dar daniilovich"

In [35]:
full_names = sirena_export_fixed.PaxName
full_names

0                 ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ
1               КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ
2          ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА
3                 ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ
4              НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ
                        ...               
155752    ЕМЕЛЬЯНОВ МИРОСЛАВ СВЯТОСЛАВОВИЧ
155753          ТАРАСОВ АРТЕМИЙ ГЕРМАНОВИЧ
155754          ГОРОХОВ БОРИС СВЯТОГОРОВИЧ
155755       ДОРОФЕЕВ РАМИЛЬ СВЯТОСЛАВОВИЧ
155756        ШАПОВАЛОВА ВАРВАРА ДАНИЛОВНА
Name: PaxName, Length: 155757, dtype: object

In [36]:
first_names = []
second_names = []
last_names = []
for full_name in full_names:
    full_name_list = full_name.strip().split()
    if len(full_name_list) == 3:
        last_name, first_name, second_name = full_name_list
        first_names.append(first_name)
        second_names.append(second_name)
        last_names.append(last_name)
    elif len(full_name_list) == 2:
        last_name, first_name = full_name_list
        first_names.append(first_name)
        second_names.append("")
        last_names.append(last_name)
    else: 
        first_names.append("")
        second_names.append("")
        last_names.append("")

In [37]:
sirena_export_fixed['first_name_rus'] = list(map(lambda x: x.lower(), first_names))
sirena_export_fixed['second_name_rus'] = list(map(lambda x: x.lower(), second_names))
sirena_export_fixed['last_name_rus'] = list(map(lambda x: x.lower(), last_names))

first_names = list(map(lambda x: x.lower(), first_names))
second_names = list(map(lambda x: x.lower(), second_names))
last_names = list(map(lambda x: x.lower(), last_names))

In [38]:
def map_rus_to_eng(name):
    return "".join(map(lambda x: rus2eng.get(x, ''), list(name)))

In [39]:
map_rus_to_eng("ильдар")

"il'dar"

In [40]:
first_names[0]

'ильдар'

In [41]:
first_names_eng = list(map(map_rus_to_eng, first_names))
second_names_eng = list(map(map_rus_to_eng, second_names))
last_names_eng = list(map(map_rus_to_eng, last_names))

sirena_export_fixed['first_name_eng'] = first_names_eng
sirena_export_fixed['second_name_eng'] = second_names_eng
sirena_export_fixed['last_name_eng'] = last_names_eng

In [42]:
sirena_export_fixed.head()

,PaxName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,...,Fare,Baggage,PaxAdditionalInfo,AgentInfo,first_name_rus,second_name_rus,last_name_rus,first_name_eng,second_name_eng,last_name_eng
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,...,JGRPGN,0PC,S FF#SU 38116280,Go2See,ильдар,даниилович,озеров,il'dar,daniilovich,ozerov
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,...,YRSTUQ,NaN,9 FF#FB 284903754,NaN,самир,тамерланович,колосов,samir,tamerlanovich,kolosov
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,...,YSTNJL,NaN,F,KupiBilet,снежана,константиновна,игнатова,snezhana,konstantinovna,ignatova
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,JFLXLS,0PC,# FF#FB 884556993,Travelgenio,платон,альбертович,жаров,platon,al'bertovich,zharov
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180,NO,SVO,VOG,...,YFLXPG,NaN,FF#SU 183142068,OZON.travel,николай,игоревич,никольский,nikolai,igorevich,nikol'skii


In [43]:
name_eng = []
last_names_eng = sirena_export_fixed.last_name_eng
first_names_eng = sirena_export_fixed.first_name_eng
second_names_eng = sirena_export_fixed.second_name_eng
for last_name, first_name, second_name in zip(last_names_eng, first_names_eng, second_names_eng):
    name_eng.append(last_name + " " + first_name + " " + second_name[0] + ".")
    
sirena_export_fixed["last_first_name_eng"] = name_eng

In [44]:
sirena_export_fixed.head(3)

,PaxName,PaxBirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,Flight,CodeSh,From,Dest,...,Baggage,PaxAdditionalInfo,AgentInfo,first_name_rus,second_name_rus,last_name_rus,first_name_eng,second_name_eng,last_name_eng,last_first_name_eng
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306,NO,SVO,OVB,...,0PC,S FF#SU 38116280,Go2See,ильдар,даниилович,озеров,il'dar,daniilovich,ozerov,ozerov il'dar d.
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323,NO,MMK,SVO,...,NaN,9 FF#FB 284903754,NaN,самир,тамерланович,колосов,samir,tamerlanovich,kolosov,kolosov samir t.
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481,NO,KJA,SVO,...,NaN,F,KupiBilet,снежана,константиновна,игнатова,snezhana,konstantinovna,ignatova,ignatova snezhana k.


In [45]:
sirena_export_fixed['last_first_name_eng'] = list(map(lambda x: x.lower(), sirena_export_fixed.last_first_name_eng))

In [46]:
name_eng = []
last_names_eng = boarding_data.PassengerLastName
first_names_eng = boarding_data.PassengerFirstName
second_names_eng = boarding_data.PassengerSecondName
for last_name, first_name, second_name in zip(last_names_eng, first_names_eng, second_names_eng):
    name_eng.append(last_name + " " + first_name + " " + second_name[0] + ".")
    
boarding_data["last_first_name_eng"] = name_eng

In [47]:
boarding_data.head(3)

,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination,last_first_name_eng
0,savelii,viktorovich,rusanov,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow,rusanov savelii v.
1,lev,markovich,isaev,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd,isaev lev m.
2,nikolai,i.,nikolskii,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd,nikolskii nikolai i.


In [48]:
boarding_data['last_first_name_eng'] = list(map(lambda x: x.lower(), boarding_data.last_first_name_eng))

In [49]:
print(boarding_data.shape)
print(sirena_export_fixed.shape)

(155147, 15)
(155757, 27)


In [50]:
boarding_data.columns

Index(['PassengerFirstName', 'PassengerSecondName', 'PassengerLastName',
       'PassengerSex', 'PassengerBirthDate', 'PassengerDocument',
       'BookingCode', 'TicketNumber', 'Baggage', 'FlightDate', 'FlightTime',
       'FlightNumber', 'CodeShare', 'Destination', 'last_first_name_eng'],
      dtype='object')

In [51]:
sirena_export_fixed.columns

Index(['PaxName', 'PaxBirthDate', 'DepartDate', 'DepartTime', 'ArrivalDate',
       'ArrivalTime', 'Flight', 'CodeSh', 'From', 'Dest', 'Code', 'e-Ticket',
       'TravelDoc', 'Seat', 'Meal', 'TrvCls', 'Fare', 'Baggage',
       'PaxAdditionalInfo', 'AgentInfo', 'first_name_rus', 'second_name_rus',
       'last_name_rus', 'first_name_eng', 'second_name_eng', 'last_name_eng',
       'last_first_name_eng'],
      dtype='object')

In [52]:
boarding_data.columns = list(map(lambda x: x + "_1", boarding_data.columns))
sirena_export_fixed.columns = list(map(lambda x: x + "_4", sirena_export_fixed.columns))

In [53]:
boarding_data.rename(columns={"last_first_name_eng_1": "FullName"}, inplace=True)
sirena_export_fixed.rename(columns={"last_first_name_eng_4": "FullName"}, inplace=True)

In [54]:
boarding_data.rename(columns={"FlightNumber_1": "Flight"}, inplace=True)
sirena_export_fixed.rename(columns={"Flight_4": "Flight"}, inplace=True)

In [55]:
boarding_data.rename(columns={"FlightDate_1": "flightDate"}, inplace=True)
sirena_export_fixed.rename(columns={"DepartDate_4": "flightDate"}, inplace=True)

In [56]:
boarding_data.columns

Index(['PassengerFirstName_1', 'PassengerSecondName_1', 'PassengerLastName_1',
       'PassengerSex_1', 'PassengerBirthDate_1', 'PassengerDocument_1',
       'BookingCode_1', 'TicketNumber_1', 'Baggage_1', 'flightDate',
       'FlightTime_1', 'Flight', 'CodeShare_1', 'Destination_1', 'FullName'],
      dtype='object')

In [57]:
sirena_export_fixed.columns

Index(['PaxName_4', 'PaxBirthDate_4', 'flightDate', 'DepartTime_4',
       'ArrivalDate_4', 'ArrivalTime_4', 'Flight', 'CodeSh_4', 'From_4',
       'Dest_4', 'Code_4', 'e-Ticket_4', 'TravelDoc_4', 'Seat_4', 'Meal_4',
       'TrvCls_4', 'Fare_4', 'Baggage_4', 'PaxAdditionalInfo_4', 'AgentInfo_4',
       'first_name_rus_4', 'second_name_rus_4', 'last_name_rus_4',
       'first_name_eng_4', 'second_name_eng_4', 'last_name_eng_4', 'FullName'],
      dtype='object')

In [58]:
tables_1_4_merged = boarding_data.merge(sirena_export_fixed, how='inner')
tables_1_4_merged

,PassengerFirstName_1,PassengerSecondName_1,PassengerLastName_1,PassengerSex_1,PassengerBirthDate_1,PassengerDocument_1,BookingCode_1,TicketNumber_1,Baggage_1,flightDate,...,Fare_4,Baggage_4,PaxAdditionalInfo_4,AgentInfo_4,first_name_rus_4,second_name_rus_4,last_name_rus_4,first_name_eng_4,second_name_eng_4,last_name_eng_4
0,artem,eduardovich,kapustin,Male,10/24/1982,0058 142289,JJADFB,7467749130398378,Transit,2017-03-12,...,YFLXNM,2PC,F,KupiBilet,артём,эдуардович,капустин,artem,eduardovich,kapustin
1,zarina,e.,titova,Female,12/22/2000,6600 251370,Not presented,5954073786122008,None,2017-07-29,...,YSTNXJ,1PC,8 FF#FB 933538031,OZON.travel,зарина,эмильевна,титова,zarina,emil'evna,titova
2,samir,gordeevich,barsukov,Male,02/21/1995,0078 271703,WGKZTB,2264717979478322,Registered,2017-07-29,...,YGRPPP,1PC,F,Aeroflot,самир,гордеевич,барсуков,samir,gordeevich,barsukov
3,denis,germanovich,sukhanov,Male,11/01/1971,5057 285380,LFDJDS,8861344450467602,None,2017-02-14,...,YSTNKL,0PC,#,Kiwi.com,денис,германович,суханов,denis,germanovich,sukhanov
4,kristina,m.,gorbunova,Female,01/29/1970,4552 046627,DMNOJI,2209898944972989,Registered,2017-11-13,...,YRSTGM,NaN,F,KupiBilet,кристина,матвеевна,горбунова,kristina,matveevna,gorbunova
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46295,milena,demidovna,bocharova,Female,12/12/1983,2930 564092,Not presented,0888291550083988,None,2017-01-08,...,AGRPHM,1PC,#,Travelgenio,милена,демидовна,бочарова,milena,demidovna,bocharova
46296,georgii,s.,ozerov,Male,06/19/1987,3242 441269,Not presented,9975783952109986,Registered,2017-01-08,...,YRSTAX,0PC,NaN,Kiwi.com,георгий,сергеевич,озеров,georgii,sergeevich,ozerov
46297,amina,s.,suslova,Female,11/27/1988,0781 620871,WKTSFS,7140622416563329,Transit,2017-02-14,...,YGRPXA,0PC,F,Kiwi.com,амина,самировна,суслова,amina,samirovna,suslova
46298,artemii,germanovich,tarasov,Male,11/28/1990,2199 078129,Not presented,Not presented,None,2017-02-14,...,YFLXKV,1PC,T FF#DT 261604825,OZON.travel,артемий,германович,тарасов,artemii,germanovich,tarasov


# Далее сагрегируем датафрейм 2 с данными из таблицы 3 (xml)
## Парсинг xml файла написан отдельно

In [59]:
pointz_aggregator_airlines = pd.read_csv("PointzAggregator-AirlinesData_3.csv", sep=';')
pointz_aggregator_airlines

,uid_3,firstName_3,lastName_3,cardNumber_3,bonusProgramm_3,flightCode_3,flightDate_3,flightDeparture_3,flightArrival_3,flightFare_3
0,613142142,iaromir,zverev,FB 171388778,Flying Blue,KE827,2017-08-06,REA,SZX,YGRPZT
1,613142142,iaromir,zverev,FB 171388778,Flying Blue,MU9706,2017-10-26,PEK,BSD,YSTNYV
2,103197717,vitalina,korovina,KE 696768759,Korean Air SKYPASS,DL5058,2017-09-11,CHA,ATL,YRSTKY
3,103197717,vitalina,korovina,KE 696768759,Korean Air SKYPASS,KE1,2017-04-01,REA,HNL,PRSTRU
4,103197717,vitalina,korovina,KE 696768759,Korean Air SKYPASS,DL837,2017-09-13,ATL,HNL,YGRPNG
...,...,...,...,...,...,...,...,...,...,...
290253,293103570,emil,kazakov,KE 637084941,Korean Air SKYPASS,AR2805,2017-11-28,FTE,AEP,YFLXMR
290254,293103570,emil,kazakov,KE 637084941,Korean Air SKYPASS,SU6401,2017-02-14,LED,SVX,YFLXZR
290255,265781820,ul'yana,kononova,DT 302785701,Delta SkyMiles,GA502,2017-05-12,CGK,PNK,YFLXAU
290256,265781820,ul'yana,kononova,DT 302785701,Delta SkyMiles,GA88,2017-08-02,CGK,AMS,YRSTQC


# Датафрейм 2 (json)

In [60]:
frequent_flyer_forum_profiles = pd.read_csv("FrequentFlyerForum-Profiles.csv", sep=";")
frequent_flyer_forum_profiles

,nickName_2,firstName_2,lastName_2,sex_2,cardNumber_2,date_2,arrival_2,flight_2,departure_2
0,FrequentFlyer0,NaN,NaN,Male,KE 889215424,2017-03-04,PIT,DL3377,BOS
1,FrequentFlyer0,NaN,NaN,Male,KE 889215424,2017-07-21,DTW,DL3630,GRB
2,FrequentFlyer0,NaN,NaN,Male,KE 889215424,2017-03-22,LGA,DL5361,PIT
3,FrequentFlyer0,NaN,NaN,Male,KE 889215424,2017-01-04,WAW,SU3702,SVO
4,FrequentFlyer0,NaN,NaN,Male,KE 889215424,2017-08-08,ATL,DL3412,XNA
...,...,...,...,...,...,...,...,...,...
1747126,FrequentFlyer99996,NaN,NaN,Male,DT 702680548,2017-12-19,PEK,MU5155,SHA
1747127,FrequentFlyer99996,NaN,NaN,Male,DT 702680548,2017-09-06,PEK,CZ6101,SHE
1747128,FrequentFlyer99996,NaN,NaN,Male,DT 702680548,2017-10-27,SHA,MU5118,PEK
1747129,FrequentFlyer99996,NaN,NaN,Male,DT 702680548,2017-06-26,SVO,SU1185,VOG


In [61]:
pointz_aggregator_airlines.rename(columns={'cardNumber_3': 'cardNumber', 'flightCode_3': 'Flight'}, inplace=True)

In [62]:
pointz_aggregator_airlines.rename(columns={'flightDate_3': 'flightDate'}, inplace=True)

In [63]:
frequent_flyer_forum_profiles.rename(columns={'cardNumber_2': 'cardNumber', 'flight_2': 'Flight'}, inplace=True)

In [64]:
frequent_flyer_forum_profiles.rename(columns={'date_2': 'flightDate'}, inplace=True)

In [65]:
tables_2_3_merged = frequent_flyer_forum_profiles.merge(pointz_aggregator_airlines, how='inner')

In [66]:
# По карте лояльности и номеру рейса и дате полета
tables_2_3_merged

,nickName_2,firstName_2,lastName_2,sex_2,cardNumber,flightDate,arrival_2,Flight,departure_2,uid_3,firstName_3,lastName_3,bonusProgramm_3,flightDeparture_3,flightArrival_3,flightFare_3
0,FrequentFlyer7,NaN,NaN,Male,DT 980250352,2017-11-11,ORY,AF4445,AJA,138879468,danil,vavilov,Delta SkyMiles,AJA,ORY,YSTNMG
1,FrequentFlyer7,NaN,NaN,Male,DT 980250352,2017-03-07,PRG,AZ7545,FCO,138879468,danil,vavilov,Delta SkyMiles,FCO,PRG,ASTNZL
2,FrequentFlyer7,NaN,NaN,Male,DT 980250352,2017-09-17,ORY,AF7331,MLH,138879468,danil,vavilov,Delta SkyMiles,MLH,ORY,YSTNOE
3,FrequentFlyer7,NaN,NaN,Male,DT 980250352,2017-10-07,AJA,AF4102,ORY,138879468,danil,vavilov,Delta SkyMiles,ORY,AJA,YSTNJD
4,FrequentFlyer10,NaN,NaN,Male,KE 172720439,2017-03-08,BUD,OK796,PRG,513348472,iaroslav,akimov,Korean Air SKYPASS,PRG,BUD,AGRPBH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140039,FrequentFlyer99995,STEFANIYA,BONDAREVA,Female,DT 406967834,2017-12-15,SJC,DL435,SLC,280456697,stefaniia,bondareva,Delta SkyMiles,SLC,SJC,YSTNZA
140040,FrequentFlyer99995,STEFANIYA,BONDAREVA,Female,DT 406967834,2017-03-29,CDG,AF4855,SVO,280456697,stefaniia,bondareva,Delta SkyMiles,SVO,CDG,YSTNOV
140041,FrequentFlyer99995,STEFANIYA,BONDAREVA,Female,DT 406967834,2017-07-01,SLC,DL1189,LAX,280456697,stefaniia,bondareva,Delta SkyMiles,LAX,SLC,AFLXNJ
140042,FrequentFlyer99995,STEFANIYA,BONDAREVA,Female,DT 406967834,2017-07-11,MCO,DL750,ATL,280456697,stefaniia,bondareva,Delta SkyMiles,ATL,MCO,YRSTRH


In [67]:
# По ФИО и номеру рейса
tables_1_4_merged

,PassengerFirstName_1,PassengerSecondName_1,PassengerLastName_1,PassengerSex_1,PassengerBirthDate_1,PassengerDocument_1,BookingCode_1,TicketNumber_1,Baggage_1,flightDate,...,Fare_4,Baggage_4,PaxAdditionalInfo_4,AgentInfo_4,first_name_rus_4,second_name_rus_4,last_name_rus_4,first_name_eng_4,second_name_eng_4,last_name_eng_4
0,artem,eduardovich,kapustin,Male,10/24/1982,0058 142289,JJADFB,7467749130398378,Transit,2017-03-12,...,YFLXNM,2PC,F,KupiBilet,артём,эдуардович,капустин,artem,eduardovich,kapustin
1,zarina,e.,titova,Female,12/22/2000,6600 251370,Not presented,5954073786122008,None,2017-07-29,...,YSTNXJ,1PC,8 FF#FB 933538031,OZON.travel,зарина,эмильевна,титова,zarina,emil'evna,titova
2,samir,gordeevich,barsukov,Male,02/21/1995,0078 271703,WGKZTB,2264717979478322,Registered,2017-07-29,...,YGRPPP,1PC,F,Aeroflot,самир,гордеевич,барсуков,samir,gordeevich,barsukov
3,denis,germanovich,sukhanov,Male,11/01/1971,5057 285380,LFDJDS,8861344450467602,None,2017-02-14,...,YSTNKL,0PC,#,Kiwi.com,денис,германович,суханов,denis,germanovich,sukhanov
4,kristina,m.,gorbunova,Female,01/29/1970,4552 046627,DMNOJI,2209898944972989,Registered,2017-11-13,...,YRSTGM,NaN,F,KupiBilet,кристина,матвеевна,горбунова,kristina,matveevna,gorbunova
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46295,milena,demidovna,bocharova,Female,12/12/1983,2930 564092,Not presented,0888291550083988,None,2017-01-08,...,AGRPHM,1PC,#,Travelgenio,милена,демидовна,бочарова,milena,demidovna,bocharova
46296,georgii,s.,ozerov,Male,06/19/1987,3242 441269,Not presented,9975783952109986,Registered,2017-01-08,...,YRSTAX,0PC,NaN,Kiwi.com,георгий,сергеевич,озеров,georgii,sergeevich,ozerov
46297,amina,s.,suslova,Female,11/27/1988,0781 620871,WKTSFS,7140622416563329,Transit,2017-02-14,...,YGRPXA,0PC,F,Kiwi.com,амина,самировна,суслова,amina,samirovna,suslova
46298,artemii,germanovich,tarasov,Male,11/28/1990,2199 078129,Not presented,Not presented,None,2017-02-14,...,YFLXKV,1PC,T FF#DT 261604825,OZON.travel,артемий,германович,тарасов,artemii,germanovich,tarasov


In [68]:
tables_1_4_merged.iloc[0]

PassengerFirstName_1                         artem
PassengerSecondName_1                  eduardovich
PassengerLastName_1                       kapustin
PassengerSex_1                                Male
PassengerBirthDate_1                    10/24/1982
PassengerDocument_1                    0058 142289
BookingCode_1                               JJADFB
TicketNumber_1                    7467749130398378
Baggage_1                                  Transit
flightDate                              2017-03-12
FlightTime_1                                 11:45
Flight                                      SU6284
CodeShare_1                                    Own
Destination_1                               Moscow
FullName                         kapustin artem e.
PaxName_4                КАПУСТИН АРТЁМ ЭДУАРДОВИЧ
PaxBirthDate_4                          1982-10-24
DepartTime_4                                 11:45
ArrivalDate_4                           2017-03-12
ArrivalTime_4                  

In [69]:
tables_2_3_merged.iloc[0]

nickName_2           FrequentFlyer7
firstName_2                     NaN
lastName_2                      NaN
sex_2                          Male
cardNumber             DT 980250352
flightDate               2017-11-11
arrival_2                       ORY
Flight                       AF4445
departure_2                     AJA
uid_3                     138879468
firstName_3                   danil
lastName_3                  vavilov
bonusProgramm_3      Delta SkyMiles
flightDeparture_3               AJA
flightArrival_3                 ORY
flightFare_3                 YSTNMG
Name: 0, dtype: object

In [70]:
tables_1_4_merged.rename(columns={'PassengerFirstName_1':'FirstName', 'PassengerLastName_1': 'LastName'}, inplace=True)

In [71]:
tables_2_3_merged.rename(columns={'firstName_3': 'FirstName', 'lastName_3': 'LastName'}, inplace=True)

In [72]:
tables_1_4_merged.columns

Index(['FirstName', 'PassengerSecondName_1', 'LastName', 'PassengerSex_1',
       'PassengerBirthDate_1', 'PassengerDocument_1', 'BookingCode_1',
       'TicketNumber_1', 'Baggage_1', 'flightDate', 'FlightTime_1', 'Flight',
       'CodeShare_1', 'Destination_1', 'FullName', 'PaxName_4',
       'PaxBirthDate_4', 'DepartTime_4', 'ArrivalDate_4', 'ArrivalTime_4',
       'CodeSh_4', 'From_4', 'Dest_4', 'Code_4', 'e-Ticket_4', 'TravelDoc_4',
       'Seat_4', 'Meal_4', 'TrvCls_4', 'Fare_4', 'Baggage_4',
       'PaxAdditionalInfo_4', 'AgentInfo_4', 'first_name_rus_4',
       'second_name_rus_4', 'last_name_rus_4', 'first_name_eng_4',
       'second_name_eng_4', 'last_name_eng_4'],
      dtype='object')

In [73]:
tables_2_3_merged.columns

Index(['nickName_2', 'firstName_2', 'lastName_2', 'sex_2', 'cardNumber',
       'flightDate', 'arrival_2', 'Flight', 'departure_2', 'uid_3',
       'FirstName', 'LastName', 'bonusProgramm_3', 'flightDeparture_3',
       'flightArrival_3', 'flightFare_3'],
      dtype='object')

In [74]:
# По фамилии, имени, номеру рейса, дате полета
tables_1_2_3_4_merged = tables_1_4_merged.merge(tables_2_3_merged, how='inner')

In [75]:
tables_1_2_3_4_merged.columns

Index(['FirstName', 'PassengerSecondName_1', 'LastName', 'PassengerSex_1',
       'PassengerBirthDate_1', 'PassengerDocument_1', 'BookingCode_1',
       'TicketNumber_1', 'Baggage_1', 'flightDate', 'FlightTime_1', 'Flight',
       'CodeShare_1', 'Destination_1', 'FullName', 'PaxName_4',
       'PaxBirthDate_4', 'DepartTime_4', 'ArrivalDate_4', 'ArrivalTime_4',
       'CodeSh_4', 'From_4', 'Dest_4', 'Code_4', 'e-Ticket_4', 'TravelDoc_4',
       'Seat_4', 'Meal_4', 'TrvCls_4', 'Fare_4', 'Baggage_4',
       'PaxAdditionalInfo_4', 'AgentInfo_4', 'first_name_rus_4',
       'second_name_rus_4', 'last_name_rus_4', 'first_name_eng_4',
       'second_name_eng_4', 'last_name_eng_4', 'nickName_2', 'firstName_2',
       'lastName_2', 'sex_2', 'cardNumber', 'arrival_2', 'departure_2',
       'uid_3', 'bonusProgramm_3', 'flightDeparture_3', 'flightArrival_3',
       'flightFare_3'],
      dtype='object')

In [76]:
people_info = tables_1_2_3_4_merged[['FirstName', 'PassengerSecondName_1', 'LastName', 'PassengerSex_1',
       'PassengerBirthDate_1', 'PassengerDocument_1', 'Baggage_1', 'flightDate', 'FlightTime_1', 'Flight',  'e-Ticket_4','CodeSh_4','Seat_4', 'From_4', 'Dest_4',
                       'TravelDoc_4', 'cardNumber']]

In [77]:
people_info

,FirstName,PassengerSecondName_1,LastName,PassengerSex_1,PassengerBirthDate_1,PassengerDocument_1,Baggage_1,flightDate,FlightTime_1,Flight,e-Ticket_4,CodeSh_4,Seat_4,From_4,Dest_4,TravelDoc_4,cardNumber
0,savva,zakharovich,kolesov,Male,09/18/1999,4123 441953,Delayed,2017-01-13,08:05,SU1459,2653095176457117,NO,NaN,NOZ,SVO,4123 441953,DT 874588705
1,adelina,bogdanovna,kolosova,Female,05/19/1973,1978 742494,Delayed,2017-01-13,08:05,SU1459,9068917014095412,NO,NaN,NOZ,SVO,1978 742494,SU 302086118
2,danila,e.,astakhov,Male,03/28/2000,4260 590460,Transit,2017-01-18,18:00,SU1253,5966129467037399,NO,NaN,SVO,NBC,4260 590460,SU 212659290
3,david,denisovich,arkhipov,Male,10/11/1977,2671 607187,Delayed,2017-02-02,12:35,SU1371,123507878719709,NO,NaN,EGO,SVO,2671 607187,SU 888737591
4,ignat,b.,kalachev,Male,06/23/1975,9297 689788,None,2017-03-09,11:15,SU1411,7102908023708213,NO,NaN,SVX,SVO,9297 689788,SU 283836907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,larisa,a.,simonova,Female,10/02/1993,6713 436381,None,2017-11-27,00:50,SU5653,8977745575894336,NO,NaN,IKT,KHV,6713 436381,SU 900130559
2935,larisa,r.,pastukhova,Female,03/29/1986,7796 073336,Transit,2017-10-30,17:40,SU1354,8940154990362639,NO,NaN,SVO,VOZ,7796 073336,DT 523598781
2936,madina,i.,gromova,Female,01/29/1974,3944 278008,None,2017-01-08,21:10,SU1337,2080511869943045,NO,NaN,ARH,SVO,3944 278008,KE 331396957
2937,artemii,germanovich,tarasov,Male,11/28/1990,2199 078129,None,2017-02-14,03:10,SU1181,6730365623280343,NO,NaN,VOG,SVO,2199 078129,DT 261604825


In [78]:
# Какие-то люди часто летают на одном самолете по разным направлениям.

In [79]:
people_info.groupby(by=['Dest_4', 'flightDate'])["FirstName", "LastName"].count().iloc[50: 100]

FirstName  LastName
Dest_4 flightDate                     
AER    2017-06-10          1         1
       2017-06-12          1         1
       2017-06-23          1         1
       2017-07-05          1         1
       2017-07-17          1         1
       2017-07-23          1         1
       2017-07-31          1         1
       2017-08-13          1         1
       2017-08-18          1         1
       2017-08-22          1         1
       2017-08-28          1         1
       2017-10-04          1         1
       2017-12-17          1         1
       2017-12-24          1         1
ARH    2017-01-09          1         1
       2017-01-30          1         1
       2017-02-05          1         1
       2017-03-13          1         1
       2017-03-14          1         1
       2017-04-03          1         1
       2017-04-13          1         1
       2017-05-01          1         1
       2017-07-05          1         1
       2017-07-28          1         1
       2017-08-07          1         1
ASF    2017-01-09          1         1
       2017-01-26          1         1
       2017-02-01          1         1
       2017-02-04          1         1
       2017-02-05          1         1
       2017-02-09          1         1
       2017-03-05          1         1
       2017-03-14          1         1
       2017-03-25          1         1
       2017-05-10          1         1
       2017-09-13          1         1
       2017-12-31          1         1
BAX    2017-01-10          1         1
       2017-01-16          2         2
       2017-02-21          1         1
       2017-04-13          1         1
       2017-04-17          1         1
       2017-05-23          1         1
CEK    2017-01-01          1         1
       2017-01-12          1         1
       2017-01-18          1         1
       2017-01-19          1         1
       2017-02-01          2         2
       2017-02-02          1         1
       2017-02-08          1         1

In [80]:
people_info.to_csv("all_joined.csv", sep=";", index=False)

In [81]:
people_info.columns

Index(['FirstName', 'PassengerSecondName_1', 'LastName', 'PassengerSex_1',
       'PassengerBirthDate_1', 'PassengerDocument_1', 'Baggage_1',
       'flightDate', 'FlightTime_1', 'Flight', 'e-Ticket_4', 'CodeSh_4',
       'Seat_4', 'From_4', 'Dest_4', 'TravelDoc_4', 'cardNumber'],
      dtype='object')

In [82]:
boarding_pass_dot_aero_data = pd.read_csv("many_files.csv")

In [83]:
boarding_pass_dot_aero_data.iloc[1].values

array([1, 1, 'MRS', 'LIDIYA ZHDANOVA', nan, nan, nan, nan, nan, 'Y', nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan], dtype=object)

In [84]:
len(boarding_pass_dot_aero_data)/365

12.032876712328767

In [87]:
zip_df = pd.read_csv("YourBoardingData.csv")
zip_df

,Unnamed: 0,FirstName_7,LastName_7,flight_7,From_7,Dest_7,flightDate_7,flightTime_7,pnr_7,e_Ticket_7
0,0,LIDIYA,ZHDANOVA,SU5436,VVO,REA,2017-01-01,11:25,DYELAO,7100246988860481
1,1,SVIATOSLAV,SOKOLOV,SU5436,VVO,REA,2017-01-01,11:25,JWARRD,7608904175905333
2,2,SVIATOSLAV,SUKHANOV,SU5436,VVO,REA,2017-01-01,11:25,SETJJH,527257587351371
3,3,DENIS,ZELENIN,SU5436,VVO,REA,2017-01-01,11:25,MZZNJI,3696048365585673
4,4,OSTROVSKAYA,IRINA,SU5436,VVO,REA,2017-01-01,11:25,NJNNPC,7470610229847684
...,...,...,...,...,...,...,...,...,...,...
615766,615766,KORNILOV,VLADIMIR,AZ7240,MXP,JED,2018-01-01,11:00,BAJUBQ,8673981301035301
615767,615767,NELLI,A,DL1161,DAY,ATL,2018-01-01,14:17,SXQFLS,6057453743742357
615768,615768,KOLPAKOVA,MARIYA,SU1502,SVO,TJM,2018-01-01,22:30,VMNBHI,4424048314480896
615769,615769,ALINA,IGNATOVA,GA312,CGK,SUB,2018-01-01,11:05,LTVLUP,3825472788916248


In [88]:
people_info

,FirstName,PassengerSecondName_1,LastName,PassengerSex_1,PassengerBirthDate_1,PassengerDocument_1,Baggage_1,flightDate,FlightTime_1,Flight,e-Ticket_4,CodeSh_4,Seat_4,From_4,Dest_4,TravelDoc_4,cardNumber
0,savva,zakharovich,kolesov,Male,09/18/1999,4123 441953,Delayed,2017-01-13,08:05,SU1459,2653095176457117,NO,NaN,NOZ,SVO,4123 441953,DT 874588705
1,adelina,bogdanovna,kolosova,Female,05/19/1973,1978 742494,Delayed,2017-01-13,08:05,SU1459,9068917014095412,NO,NaN,NOZ,SVO,1978 742494,SU 302086118
2,danila,e.,astakhov,Male,03/28/2000,4260 590460,Transit,2017-01-18,18:00,SU1253,5966129467037399,NO,NaN,SVO,NBC,4260 590460,SU 212659290
3,david,denisovich,arkhipov,Male,10/11/1977,2671 607187,Delayed,2017-02-02,12:35,SU1371,123507878719709,NO,NaN,EGO,SVO,2671 607187,SU 888737591
4,ignat,b.,kalachev,Male,06/23/1975,9297 689788,None,2017-03-09,11:15,SU1411,7102908023708213,NO,NaN,SVX,SVO,9297 689788,SU 283836907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,larisa,a.,simonova,Female,10/02/1993,6713 436381,None,2017-11-27,00:50,SU5653,8977745575894336,NO,NaN,IKT,KHV,6713 436381,SU 900130559
2935,larisa,r.,pastukhova,Female,03/29/1986,7796 073336,Transit,2017-10-30,17:40,SU1354,8940154990362639,NO,NaN,SVO,VOZ,7796 073336,DT 523598781
2936,madina,i.,gromova,Female,01/29/1974,3944 278008,None,2017-01-08,21:10,SU1337,2080511869943045,NO,NaN,ARH,SVO,3944 278008,KE 331396957
2937,artemii,germanovich,tarasov,Male,11/28/1990,2199 078129,None,2017-02-14,03:10,SU1181,6730365623280343,NO,NaN,VOG,SVO,2199 078129,DT 261604825


In [104]:
for j, e_ticket in tqdm(enumerate(people_info["e-Ticket_4"].values)):
    for i in range(len(zip_df)):
        if (zip_df['e_Ticket_7'].iloc[i] == e_ticket) and ((zip_df.LastName_7.iloc[i].upper() != people_info.iloc[j].LastName.upper()) and (zip_df.LastName_7.iloc[i].upper() != people_info.iloc[j].FirstName.upper())):
            print(zip_df.iloc[i])
            print(people_info[people_info['e-Ticket_4'] == e_ticket])








0it [00:00, ?it/s]






1it [00:05,  5.88s/it]






2it [00:12,  6.05s/it]






3it [00:20,  6.60s/it]






4it [00:26,  6.39s/it]






5it [00:32,  6.40s/it]






6it [00:38,  6.23s/it]






7it [00:44,  6.20s/it]






8it [00:50,  6.16s/it]






9it [00:56,  6.12s/it]






10it [01:02,  6.16s/it]






11it [01:08,  6.11s/it]






12it [01:14,  6.12s/it]






13it [01:20,  6.04s/it]






14it [01:27,  6.15s/it]






15it [01:33,  6.15s/it]






16it [01:40,  6.31s/it]






17it [01:46,  6.27s/it]






18it [01:52,  6.18s/it]






19it [01:58,  6.18s/it]






20it [02:04,  6.11s/it]






21it [02:10,  6.12s/it]






22it [02:16,  6.10s/it]






23it [02:22,  6.03s/it]






24it [02:28,  6.09s/it]






25it [02:34,  6.08s/it]






26it [02:40,  6.14s/it]

Unnamed: 0                 43164
FirstName_7                MARAT
LastName_7                     N
flight_7                  SU1260
From_7                       SVO
Dest_7                       UFA
flightDate_7          2017-01-26
flightTime_7               16:55
pnr_7                     MWOBWX
e_Ticket_7      4957928731881295
Name: 43164, dtype: object
   FirstName PassengerSecondName_1   LastName PassengerSex_1  \
26     marat                    n.  konovalov           Male   

   PassengerBirthDate_1 PassengerDocument_1 Baggage_1  flightDate  \
26           12/06/1977         1941 534364   Transit  2017-01-26   

   FlightTime_1  Flight        e-Ticket_4 CodeSh_4  Seat_4 From_4 Dest_4  \
26        16:55  SU1260  4957928731881295       NO     NaN    SVO    UFA   

    TravelDoc_4    cardNumber  
26  1941 534364  FB 251060301  









27it [02:47,  6.21s/it]






28it [02:54,  6.57s/it]






29it [03:01,  6.59s/it]






30it [03:09,  6.96s/it]






31it [03:16,  7.07s/it]






32it [03:22,  6.82s/it]






33it [03:29,  6.84s/it]






34it [03:36,  6.82s/it]






35it [03:43,  6.85s/it]






36it [03:51,  7.13s/it]






37it [03:59,  7.47s/it]






38it [04:05,  7.04s/it]






39it [04:13,  7.35s/it]






40it [04:19,  7.05s/it]






41it [04:26,  6.84s/it]






42it [04:32,  6.76s/it]






43it [04:40,  6.95s/it]






44it [04:46,  6.70s/it]






45it [04:53,  6.77s/it]






46it [04:59,  6.72s/it]






47it [05:05,  6.54s/it]

Unnamed: 0                106381
FirstName_7                 NINA
LastName_7                     V
flight_7                  SU1508
From_7                       SVO
Dest_7                       TJM
flightDate_7          2017-03-05
flightTime_7               00:30
pnr_7                     HWAOAH
e_Ticket_7      8987986269344566
Name: 106381, dtype: object
   FirstName PassengerSecondName_1    LastName PassengerSex_1  \
47      nina             vadimovna  paramonova         Female   

   PassengerBirthDate_1 PassengerDocument_1 Baggage_1  flightDate  \
47           11/06/1991         9113 208907   Transit  2017-03-05   

   FlightTime_1  Flight        e-Ticket_4 CodeSh_4  Seat_4 From_4 Dest_4  \
47        00:30  SU1508  8987986269344566       NO     NaN    SVO    TJM   

    TravelDoc_4    cardNumber  
47  9113 208907  KE 631078735  









48it [05:12,  6.56s/it]






49it [05:18,  6.41s/it]






50it [05:24,  6.32s/it]






51it [05:31,  6.31s/it]






52it [05:37,  6.27s/it]

...








2925it [5:23:18,  6.68s/it]






2926it [5:23:25,  6.74s/it]






2927it [5:23:31,  6.67s/it]






2928it [5:23:38,  6.61s/it]






2929it [5:23:44,  6.60s/it]

Unnamed: 0                232579
FirstName_7            EKATERINA
LastName_7                     V
flight_7                  SU1439
From_7                       UFA
Dest_7                       SVO
flightDate_7          2017-05-19
flightTime_7               14:15
pnr_7                     DTXXMB
e_Ticket_7      9437237243795039
Name: 232579, dtype: object
      FirstName PassengerSecondName_1  LastName PassengerSex_1  \
2929  ekaterina             vasilevna  fedorova         Female   

     PassengerBirthDate_1 PassengerDocument_1   Baggage_1  flightDate  \
2929           03/09/1995         9075 653058  Registered  2017-05-19   

     FlightTime_1  Flight        e-Ticket_4 CodeSh_4  Seat_4 From_4 Dest_4  \
2929        14:15  SU1439  9437237243795039       NO     NaN    UFA    SVO   

      TravelDoc_4    cardNumber  
2929  9075 653058  FB 384976654  









2930it [5:23:51,  6.56s/it]

Unnamed: 0                 16068
FirstName_7               ANDREY
LastName_7                     V
flight_7                  SU1491
From_7                       SLY
Dest_7                       SVO
flightDate_7          2017-01-08
flightTime_7               13:20
pnr_7                     HGKTWJ
e_Ticket_7      4046977806572252
Name: 16068, dtype: object
     FirstName PassengerSecondName_1 LastName PassengerSex_1  \
2930    andrei                    v.    fokin           Male   

     PassengerBirthDate_1 PassengerDocument_1   Baggage_1  flightDate  \
2930           11/21/1983         8612 238111  Registered  2017-01-08   

     FlightTime_1  Flight        e-Ticket_4 CodeSh_4  Seat_4 From_4 Dest_4  \
2930        13:20  SU1491  4046977806572252       NO     NaN    SLY    SVO   

      TravelDoc_4    cardNumber  
2930  8612 238111  DT 156087373  









2931it [5:23:57,  6.45s/it]






2932it [5:24:03,  6.46s/it]






2933it [5:24:10,  6.49s/it]






2934it [5:24:16,  6.53s/it]






2935it [5:24:23,  6.48s/it]






2936it [5:24:29,  6.53s/it]






2937it [5:24:36,  6.57s/it]






2938it [5:24:43,  6.55s/it]






2939it [5:24:49,  6.63s/it]
